# import lib

In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as f
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [2]:
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

In [3]:
device

device(type='cpu')

# 1 - Data Preprocessing

In [4]:
#line file  
lines_filepath = os.path.join("cornell movie-dialogs corpus","movie_lines.txt")
#conversation file
con_filepath = os.path.join("cornell movie-dialogs corpus","movie_conversations.txt")

PROCESS THE DATA FROM LINES FILE

In [5]:
#visualize some lines
with open(lines_filepath,'r') as file:
    lines =file.readlines()
for line in lines[:8]:
    print(line.strip())

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No


In [19]:
#split each line of the file into a dictionary of feilds (lineId,characterId,movieId,character,text)

line_fields = ["lineId","characterId","movieId","character","text"]
lines ={}
#encode in ASCII
with open(lines_filepath,'r',encoding='iso-8859-1') as f:
    for line in f:
        values = line.split(' +++$+++ ')
        #extract fields
        lineObj = {}
        for i, field in enumerate(line_fields):
            #lineObj is dictionary which contaion all lines_fields as key and values
            lineObj[field] = values[i]
        #key for lines dictionary is lineIdand value is lineObj
        lines[lineObj["lineId"]] = lineObj

In [20]:
#1st element of lines dictionary
l = lines
list(l.items())[0]

('L1045',
 {'lineId': 'L1045',
  'characterId': 'u0',
  'movieId': 'm0',
  'character': 'BIANCA',
  'text': 'They do not!\n'})

PROCESS THE DATA FROM CONVERSATION FILE

In [24]:
#visualize some conversation file
with open(con_filepath,'r') as file:
    convs =file.readlines()
for line in convs[:8]:
    print(line.strip())

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']


In [ ]:
#split each onversation of the file into a dictionary of feilds ("character1Id","character2Id","movieId","utteranceIds")

conv_fields = ["character1Id","character2Id","movieId","utteranceIds"]
connversations =[]
#encode in ASCII
with open(con_filepath,'r',encoding='iso-8859-1') as f:
    for line in f:
        values = line.split(' +++$+++ ')
#         #extract fields
        convObj = {}
        for i, field in enumerate(conv_fields):
#             #lineObj is dictionary which contaion all lines_fields as key and values
            convObj[field] = values[i]
#     #split str result from split to list of movieIds
        lineIds = eval(convObj["utteranceIds"])
#     #reassemble lines
        convObj["lines"] = []
        for lineId in lineIds:
            print(lines[0])
#             convObj["lines"].append(lines[lineId])
#             conversations.append(convObj)